In [29]:

# Add the parent directory to the path so we can import the module
path_to_add = os.path.split(os.getcwd())[0]
if path_to_add not in sys.path:
    sys.path.append(path_to_add)

import yaml
import os
import sys
from src import sim_util
import glob
import datetime
import sympy as sp
from src import hw_symbols

def get_latest_log_dir():
    log_dirs = glob.glob("../logs/*-*-*_*-*-*")
    log_dirs = sorted(log_dirs, key=lambda x: datetime.datetime.strptime(x.split("/")[-1], "%Y-%m-%d_%H-%M-%S"))
    return log_dirs[-1]

log_dir = get_latest_log_dir()

print(log_dir)

log_file = open(f"{log_dir}/log.txt", 'r')
log_lines = log_file.readlines()
i = 0
for line in log_lines:
    if line.startswith("INFO:src.hardwareModel:BUFFER") or line.startswith("INFO:src.hardwareModel:MEMORY"):
        print(line)
        i += 1
        if i == 2: break

rcs = yaml.load(open(f"{log_dir}/rcs_0.yaml", 'r'), Loader=yaml.Loader)
tech_params = sim_util.generate_init_params_from_rcs_as_symbols(rcs)
cacti_exprs_file = open(f"{log_dir}/cacti_exprs_0.txt", 'r')
cacti_lines = cacti_exprs_file.readlines()
cacti_exprs = {}
for line in cacti_lines:
    sp_expr = line[line.find(':')+2:-1]
    name = line[:line.find(':')]
    cacti_exprs[name] = sp.sympify(sp_expr)
    cacti_exprs[name] = cacti_exprs[name].subs({symbol: tech_params[hw_symbols.symbol_table[symbol.name]] for symbol in cacti_exprs[name].free_symbols}).simplify()
    print(f"{name}: {cacti_exprs[name]}")

../logs/2024-09-24_08-38-48
INFO:src.hardwareModel:BUFFER VALS: read/write time 2.90872 ns, read energy 0.138167 nJ, write energy 0.142583 nJ, leakage power 5.13338

INFO:src.hardwareModel:MEMORY VALS: read/write time 6.13262 ns, read energy 0.580929 nJ, write energy 0.938415 nJ, leakage power 52.4214

{Reff_Add: 18551639.340001922, Ceff_Add: 1.0633723572320962e-07, Reff_And: 12823433.949309705, Ceff_And: 2.1757042700330577e-08, Reff_BitAnd: 43479455.73437821, Ceff_BitAnd: 8.831757286611571e-10, Reff_BitOr: 46173957.216508694, Ceff_BitOr: 8.662891900826447e-10, Reff_BitXor: 40979387.02965147, Ceff_BitXor: 9.175344661157024e-10, Reff_Eq: 9637159.83316716, Ceff_Eq: 2.9054203193388427e-08, Reff_FloorDiv: 40150.38201482472, Ceff_FloorDiv: 0.00017305439329156364, Reff_Gt: 6463147.8594782865, Ceff_Gt: 6.451964415272727e-08, Reff_GtE: 7324287.226032434, Ceff_GtE: 5.7479995937851235e-08, Reff_Invert: 146354953.67732668, Ceff_Invert: 1.9131569719008265e-10, Reff_IsNot: 146354953.67732668, Ceff_